# Content Embedding

In [80]:
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import base64
import httpx

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

True

In [97]:
response = requests.get('https://app.daytrip.io/ko/daylog/672da1fc7a109e225c15a591')
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

name = soup.select_one('main > header > div > h1').text
location, space_type = soup.select_one('main > header > div > p').text.split(' • ')
username = soup.select_one('main > section > div > label').text
description = '\n'.join([line for line in soup.select_one('main > section > p').strings])

# download images
images_div = soup.select_one('main > header > div > div > div')
image_srcs = [img['src'] for img in images_div.find_all('img') if 'src' in img.attrs]
encoded_images = []
total_image_size = 0
for url in image_srcs:
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    
    query_params.update({ 'fmt': 'jpg', 'fit': 'outside', 'w': '500', 'h': '500' })
    
    new_query = urlencode(query_params, doseq=True)
    new_url = urlunparse(parsed_url._replace(query=new_query))

    image_data = base64.b64encode(httpx.get(new_url).content).decode("utf-8")
    total_image_size += len(image_data)
    
    encoded_images.append(image_data)
    
print(f"Total image size: {total_image_size}")

Total image size: 1464992


In [107]:
llm = ChatOpenAI(model="gpt-4o")

In [112]:
image_descriptions = []
for encoded_image in encoded_images:
    messages = [
        SystemMessage((
            "You are an AI that generates descriptions for image.\n"
            "Given a image, provide a brief description of what you see.\n"
            "\n"
            "Always respond in English."
        )),
        HumanMessage(content=[
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}},
        ]),
    ]

    response = llm.invoke(messages)
    image_descriptions.append(response.content.replace("\n", ""))

In [124]:
chain_input = {
    "name": name,
    "type": space_type,
    "location": location,
    "image_descriptions": "\n".join([f"- {description}" for description in image_descriptions]),
    "description": description,
}

In [150]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", (
            "You will be generating description of place where your descript will be used for search.\n"
            "You are encouraged to summarize images more than description provided.\n"
            "\n"
            "These are points you should focus specifically:\n"
            "- type of the place (i.e. restaurant, cafe, bakery, bookstore, etc.)\n"
            "- mood (i.e. cozy, modern, dark, etc.)\n"
            "- location\n"
            "- what the place offer"
            "- what people can do"
            "\n"
            "Please follow the below restrictions:\n"
            "- Always respond in English.\n"
            "- Write in 100-200 words.\n"
            "- Write in a descriptive rather than a suggestive tone.\n"
        )),
        ("user", (
            "These are information of the place:\n"
            "\n"
            "[Name] {name}\n"
            "[Type] {type}\n"
            "[Region] {location}\n"
            "\n"
            "[Descriptions of Place Images]\n"
            "{image_descriptions}\n"
            "\n"
            "[Description]\n"
            "{description}\n"
        ))
    ]
)

prompt.invoke(chain_input).to_messages()[-1].pretty_print()

================================ Human Message =================================

These are information of the place:

[Name] 고든램지 스트리트 버거
[Type] 음식점
[Location] 서울, 강남구

[Descriptions of Place Images]
- The image shows the entrance to a restaurant named "Street Burger" by Gordon Ramsay. The entrance features a bright sign above and has open doors leading inside. The interior has a modern design with a visible counter and seating area where people are dining. The flooring has a herringbone pattern, and there are decorative elements on the walls, including text.
- The image shows a modern restaurant interior with a large communal wooden table surrounded by black stools. The ceiling is decorated with bold, white, three-dimensional letters. In the background, there is a counter area where customers can place and pick up orders. Several people are seated at the table, enjoying their meals. The atmosphere is casual and contemporary.
- The image shows a modern, casual dining area with people 

In [156]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", (
            "You are a location description expert specializing in writing short summaries for search engines."
            "Focus on capturing the ambiance, purpose, and unique experience of each location to help users find places "
            "that match their specific needs,such as \"a quiet bar for two\" or \"a lively spot for group fun.\"\n"
            "\n"
            "Please write a concise, 100-150 words description for a location "
            "that highlights its ambiance, purpose, and unique experience, making it highly relevant to user searches.\n"
            "\n"
            "Always respond in English.\n"
            "Focus more on images than descriptions provided.\n"
        )),
        ("user", (
            "These are information of the place:\n"
            "\n"
            "[Name] {name}\n"
            "[Type] {type}\n"
            "[Region] {location}\n"
            "\n"
            "[Descriptions of Place Images]\n"
            "{image_descriptions}\n"
            "\n"
            "[Description]\n"
            "{description}\n"
        ))
    ]
)

prompt.invoke(chain_input).to_messages()[-1].pretty_print()

================================ Human Message =================================

These are information of the place:

[Name] 고든램지 스트리트 버거
[Type] 음식점
[Region] 서울, 강남구

[Descriptions of Place Images]
- The image shows the entrance to a restaurant named "Street Burger" by Gordon Ramsay. The entrance features a bright sign above and has open doors leading inside. The interior has a modern design with a visible counter and seating area where people are dining. The flooring has a herringbone pattern, and there are decorative elements on the walls, including text.
- The image shows a modern restaurant interior with a large communal wooden table surrounded by black stools. The ceiling is decorated with bold, white, three-dimensional letters. In the background, there is a counter area where customers can place and pick up orders. Several people are seated at the table, enjoying their meals. The atmosphere is casual and contemporary.
- The image shows a modern, casual dining area with people se

In [157]:
chain = prompt | llm

response = chain.invoke(chain_input)
print(response.content)

Gordon Ramsay Street Burger in Seoul's bustling Gangnam district offers a chic yet casual dining experience perfect for burger enthusiasts. Step into a modern space with bold décor and a lively atmosphere, where large communal tables invite group gatherings. This globally renowned spot, the first international branch outside the UK, serves up gourmet street burgers like the Mexican Smash Burger, alongside indulgent sides such as crispy nachos and signature fries. The menu also features delightful shakes, including the new Vanilla Shake, and sweet treats like the Oreo Chocolate Cream Pie. Ideal for casual hangouts or quick bites, this venue combines a fast-food vibe with gourmet flair, making it an excellent choice for both locals and visitors seeking quality burgers crafted by top chefs. Enjoy special promotions and a vibrant culinary experience in this contemporary eatery.
